In [ ]:
# # Pypdf extraction and OpenAI classification script
# import pandas as pd
# import pypdf
# import os
# import json
# import time
# import logging
# from openai import OpenAI # Use the updated OpenAI library import
# from typing import Dict, Any

# # === LOGGING SETUP ===
# # Sets up basic configuration for logging messages.
# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s | %(levelname)s | %(message)s"
# )
# logger = logging.getLogger(__name__)

# # === CONFIGURATION ===
# # It's better to manage paths and model names in a central place.
# # The API key is read from a local file for better security.
# API_KEY_PATH = "/Users/jake/Documents/Key/OPENAI_KEY.txt" # <--- ADJUST IF NEEDED
# MODEL = "gpt-4o" # Using the latest model as specified

# def get_openai_client(api_key_path: str) -> OpenAI:
#     """Reads the OpenAI API key from a file and returns an OpenAI client."""
#     try:
#         with open(api_key_path, 'r') as f:
#             api_key = f.read().strip()
#         if not api_key:
#             raise ValueError("API key file is empty.")
#         logger.info("Successfully loaded OpenAI API key.")
#         return OpenAI(api_key=api_key)
#     except FileNotFoundError:
#         logger.error(f"API key file not found at: {api_key_path}")
#         return None
#     except Exception as e:
#         logger.error(f"An error occurred while reading the API key: {e}")
#         return None

# def extract_text_from_pdf(pdf_path: str) -> pd.DataFrame:
#     """
#     Extracts all text from a given PDF file, page by page.
#     """
#     if not os.path.exists(pdf_path):
#         logger.error(f"Error: File not found at {pdf_path}")
#         return pd.DataFrame()

#     logger.info(f"Starting text extraction from '{os.path.basename(pdf_path)}'...")
#     all_pages_data = []

#     try:
#         with open(pdf_path, 'rb') as file:
#             reader = pypdf.PdfReader(file)
#             num_pages = len(reader.pages)
#             logger.info(f"Found {num_pages} pages in the document.")

#             for i, page in enumerate(reader.pages):
#                 page_number = i + 1
#                 text = page.extract_text() or "" # Ensure text is a string
                
#                 all_pages_data.append({
#                     'page_number': page_number,
#                     'text': text.strip()
#                 })
#                 if not text.strip():
#                     logger.info(f"  - No text found on page {page_number}.")

#     except Exception as e:
#         logger.error(f"An error occurred while processing the PDF: {e}")
#         return pd.DataFrame()

#     if all_pages_data:
#         df = pd.DataFrame(all_pages_data)
#         logger.info("Text extraction complete.")
#         return df
#     else:
#         logger.warning("Warning: No text was extracted from the document.")
#         return pd.DataFrame()

# def classify_page_text(client: OpenAI, page_number: int, page_text: str) -> Dict[str, Any]:
#     """
#     Classifies the text of a single page using the OpenAI API.
#     """
#     # The new system prompt with updated instructions and JSON format.
#     # Note: The model is only asked for classification details.
#     # Page number and text are added back into the final dictionary later.
#     sys_prompt = """
# You are an expert insurance regulatory analyst reviewing a state commercial auto insurance rate and rule filing.

# Your job is to classify each page into a single best-fitting category ("bucket"). The following buckets are examples of likely categories, but you are allowed to invent and assign a new, appropriate bucket name, if the existing examples do not fit.

# BUCKET EXAMPLES (use or invent as needed):

# - intro information: Cover letters, summaries, company info, administrative headers.
# - table of contents: Tables/indexes listing sections, rules, forms.
# - correspondence: Letters, memos, formal or informal communication (including with a regulator).
# - rule: Detailed rating rules, eligibility, underwriting guidelines, standard operating instructions.
# - factor table: Tabular lists of rating factors—e.g. for zones, territories, drivers, vehicles.
# - actuarial support: Mathematical or statistical justification, trend documentation, loss ratios, exhibits.
# - form: Complete forms, endorsements, schedules, specimen policy wordings.
# - rating example: A worked example showing how premium/rate is calculated.
# - exhibit: Graphs, charts, additional annotated attachments or appendices.
# - crossed_out: (binary) Use ONLY if the entire page is covered with a line, annotated "withdrawn," or has visible strikethrough/crossout. Otherwise, do not use.
# - other: Use only if the page fits none of the above and you cannot reasonably propose a more accurate new bucket name.

# BUCKET FLEXIBILITY:
# - If none of the above buckets are a good fit, make up an appropriate, concise, descriptive bucket name and use it as the "bucket". Do NOT use "llm_new_category" as a category name—use your proposed name directly (e.g. "signature page", "state certification", etc).

# CATEGORIZATION INSTRUCTIONS:
# - Assign exactly one bucket per page.
# - Always provide a 10 word "explanation" of why you selected—or if new, created—this bucket.
# - If "crossed_out" is chosen, no substantive explanation is needed—just state "Entire page was striked out or withdrawn."
# - Otherwise, explain the dominant content and your reasoning for the bucket chosen in precisely 10 words.

# OUTPUT FORMAT (respond with a single valid JSON object only):

# {
#   "bucket": "<bucket_name>",
#   "confidence": <probability 0-1>,
#   "explanation": "<10 word explanation of categorization>"
# }

# If uncertain, favor "other", but prefer to create (with reasoned explanation) a new appropriate bucket when justified.
# """
#     # Create the full result dictionary here, starting with known values.
#     # This ensures page_number and text are always present, even on error.
#     result_payload = {
#         "page_number": page_number,
#         "bucket": "processing_error",
#         "confidence": 0.0,
#         "explanation": "An error occurred before the API call.",
#         "text": page_text
#     }

#     if not page_text:
#         result_payload.update({
#             "bucket": "other",
#             "confidence": 1.0,
#             "explanation": "Page is blank or contains no extractable text."
#         })
#         return result_payload

#     for attempt in range(3): # Retry logic for transient API errors
#         try:
#             chat_completion = client.chat.completions.create(
#                 messages=[
#                     {"role": "system", "content": sys_prompt},
#                     {"role": "user", "content": page_text[:16000]}, # Increased token limit for gpt-4o
#                 ],
#                 model=MODEL,
#                 response_format={"type": "json_object"},
#                 temperature=0.0,
#             )
#             response_content = chat_completion.choices[0].message.content
#             # Parse the JSON from the model
#             api_result = json.loads(response_content)
#             # Update the payload with the model's response
#             result_payload.update(api_result)
#             return result_payload
#         except Exception as e:
#             logger.warning(f"API call failed on attempt {attempt + 1} for page {page_number}: {e}. Retrying in {2 ** attempt}s...")
#             time.sleep(2 ** attempt)

#     logger.error(f"API call failed after multiple retries for page {page_number}.")
#     result_payload.update({
#         "bucket": "api_error",
#         "explanation": "API call failed after multiple retries."
#     })
#     return result_payload

# # --- Main Script Execution ---
# if __name__ == "__main__":
#     # --- USER INPUT ---
#     input_pdf_path = "./Inputs/PGR_Ohio_BNIC-134120828_trimmed.pdf"
#     output_csv_path = "./Output/classified_pdf_text_aiv2.csv"

#     # --- SCRIPT LOGIC ---
#     # 1. Initialize OpenAI Client
#     openai_client = get_openai_client(API_KEY_PATH)
    
#     if openai_client:
#         # 2. Extract text from PDF
#         pdf_dataframe = extract_text_from_pdf(input_pdf_path)

#         if not pdf_dataframe.empty:
#             # 3. Classify each page
#             logger.info("Starting page classification process...")
            
#             results = []
#             total_pages = len(pdf_dataframe)
#             for index, row in pdf_dataframe.iterrows():
#                 logger.info(f"Classifying page {row['page_number']}/{total_pages}...")
#                 # Pass page number and text to the classification function
#                 result = classify_page_text(
#                     client=openai_client, 
#                     page_number=row['page_number'], 
#                     page_text=row['text']
#                 )
#                 results.append(result)

#             # 4. Create the final DataFrame from the list of result dictionaries
#             final_df = pd.DataFrame(results)

#             # 5. Save the final results to CSV
#             try:
#                 # Reorder columns for clarity in the output CSV
#                 column_order = ['page_number', 'bucket', 'confidence', 'explanation', 'text']
#                 final_df = final_df[column_order]
                
#                 final_df.to_csv(output_csv_path, index=False, encoding='utf-8')
#                 logger.info(f"\nSuccessfully saved classified text to '{output_csv_path}'")
                
#                 logger.info("\n--- Sample of Final Data (text column omitted for brevity) ---")
#                 print(final_df.drop(columns=['text']).head().to_string())

#             except Exception as e:
#                 logger.error(f"\nAn error occurred while saving the CSV file: {e}")

2025-08-26 11:02:24,105 | INFO | Successfully loaded OpenAI API key.
2025-08-26 11:02:24,133 | INFO | Starting text extraction from 'PGR_Ohio_BNIC-134120828_trimmed.pdf'...
2025-08-26 11:02:24,144 | INFO | Found 72 pages in the document.
2025-08-26 11:02:24,927 | INFO |   - No text found on page 68.
2025-08-26 11:02:24,956 | INFO | Text extraction complete.
2025-08-26 11:02:24,956 | INFO | Starting page classification process...
2025-08-26 11:02:24,958 | INFO | Classifying page 1/72...
2025-08-26 11:02:26,151 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-26 11:02:26,159 | INFO | Classifying page 2/72...
2025-08-26 11:02:27,168 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-26 11:02:27,185 | INFO | Classifying page 3/72...
2025-08-26 11:02:28,172 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-26 11:02:28,187 | INFO | Classifying page 4/72

   page_number             bucket  confidence                                                                     explanation
0            1  table of contents        0.95            Lists sections, rules, forms, and attachments in document structure.
1            2  intro information        0.90  Summary of filing requirements and administrative header information provided.
2            3  intro information        0.95        Summary of filing details, company, product, and submission information.
3            4  intro information        0.95         Contains company info, project details, and filing description summary.
4            5  intro information        0.90         Contains filing fees, company info, and administrative details summary.


In [ ]:
# Pypdf extraction and OpenAI classification script
import pandas as pd
import pypdf
import os
import json
import time
import logging
from openai import OpenAI # Use the updated OpenAI library import
from typing import Dict, Any, List

# === LOGGING SETUP ===
# Sets up basic configuration for logging messages.
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)
logger = logging.getLogger(__name__)

# === CONFIGURATION ===
# It's better to manage paths and model names in a central place.
# The API key is read from a local file for better security.
API_KEY_PATH = "/Users/jake/Documents/Key/OPENAI_KEY.txt" # <--- ADJUST IF NEEDED
MODEL = "gpt-4o" # Using the latest model as specified

def get_openai_client(api_key_path: str) -> OpenAI:
    """Reads the OpenAI API key from a file and returns an OpenAI client."""
    try:
        with open(api_key_path, 'r') as f:
            api_key = f.read().strip()
        if not api_key:
            raise ValueError("API key file is empty.")
        logger.info("Successfully loaded OpenAI API key.")
        return OpenAI(api_key=api_key)
    except FileNotFoundError:
        logger.error(f"API key file not found at: {api_key_path}")
        return None
    except Exception as e:
        logger.error(f"An error occurred while reading the API key: {e}")
        return None

def extract_text_from_pdf(pdf_path: str) -> pd.DataFrame:
    """
    Extracts all text from a given PDF file, page by page.
    """
    if not os.path.exists(pdf_path):
        logger.error(f"Error: File not found at {pdf_path}")
        return pd.DataFrame()

    logger.info(f"Starting text extraction from '{os.path.basename(pdf_path)}'...")
    all_pages_data = []

    try:
        with open(pdf_path, 'rb') as file:
            reader = pypdf.PdfReader(file)
            num_pages = len(reader.pages)
            logger.info(f"Found {num_pages} pages in the document.")

            for i, page in enumerate(reader.pages):
                page_number = i + 1
                text = page.extract_text() or "" # Ensure text is a string
                
                all_pages_data.append({
                    'page_number': page_number,
                    'text': text.strip()
                })
                if not text.strip():
                    logger.info(f"  - No text found on page {page_number}.")

    except Exception as e:
        logger.error(f"An error occurred while processing the PDF: {e}")
        return pd.DataFrame()

    if all_pages_data:
        df = pd.DataFrame(all_pages_data)
        logger.info("Text extraction complete.")
        return df
    else:
        logger.warning("Warning: No text was extracted from the document.")
        return pd.DataFrame()

def classify_page_text(client: OpenAI, page_number: int, page_text: str) -> Dict[str, Any]:
    """
    Classifies the text of a single page using the OpenAI API.
    """
    # The new system prompt with updated instructions and JSON format.
    # Note: The model is only asked for classification details.
    # Page number and text are added back into the final dictionary later.
    sys_prompt = """
You are an expert insurance regulatory analyst reviewing a state commercial auto insurance rate and rule filing.

Your job is to classify each page into a single best-fitting category ("bucket"). The following buckets are examples of likely categories, but you are allowed to invent and assign a new, appropriate bucket name, if the existing examples do not fit.

BUCKET EXAMPLES (use or invent as needed):

- intro information: Cover letters, summaries, company info, administrative headers.
- table of contents: Tables/indexes listing sections, rules, forms.
- correspondence: Letters, memos, formal or informal communication (including with a regulator).
- rule: Detailed rating rules, eligibility, underwriting guidelines, standard operating instructions.
- factor_table: Tabular lists of rating factors—e.g. for zones, territories, drivers, vehicles.
- actuarial support: Mathematical or statistical justification, trend documentation, loss ratios, exhibits.
- form: Complete forms, endorsements, schedules, specimen policy wordings.
- rating example: A worked example showing how premium/rate is calculated.
- exhibit: Graphs, charts, additional annotated attachments or appendices.
- crossed_out: (binary) Use ONLY if the entire page is covered with a line, annotated "withdrawn," or has visible strikrough/crossout. Otherwise, do not use.
- other: Use only if the page fits none of the above and you cannot reasonably propose a more accurate new bucket name.

BUCKET FLEXIBILITY:
- If none of the above buckets are a good fit, make up an appropriate, concise, descriptive bucket name and use it as the "bucket". Do NOT use "llm_new_category" as a category name—use your proposed name directly (e.g. "signature page", "state certification", etc).

CATEGORIZATION INSTRUCTIONS:
- Assign exactly one bucket per page.
- Always provide a 10 word "explanation" of why you selected—or if new, created—this bucket.
- If "crossed_out" is chosen, no substantive explanation is needed—just state "Entire page was striked out or withdrawn."
- Otherwise, explain the dominant content and your reasoning for the bucket chosen in precisely 10 words.

OUTPUT FORMAT (respond with a single valid JSON object only):

{
  "bucket": "<bucket_name>",
  "confidence": <probability 0-1>,
  "explanation": "<10 word explanation of categorization>"
}

If uncertain, favor "other", but prefer to create (with reasoned explanation) a new appropriate bucket when justified.
"""
    # Create the full result dictionary here, starting with known values.
    # This ensures page_number and text are always present, even on error.
    result_payload = {
        "page_number": page_number,
        "bucket": "processing_error",
        "confidence": 0.0,
        "explanation": "An error occurred before the API call.",
        "text": page_text
    }

    if not page_text:
        result_payload.update({
            "bucket": "other",
            "confidence": 1.0,
            "explanation": "Page is blank or contains no extractable text."
        })
        return result_payload

    for attempt in range(3): # Retry logic for transient API errors
        try:
            chat_completion = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": sys_prompt},
                    {"role": "user", "content": page_text[:16000]}, # Increased token limit for gpt-4o
                ],
                model=MODEL,
                response_format={"type": "json_object"},
                temperature=0.0,
            )
            response_content = chat_completion.choices[0].message.content
            # Parse the JSON from the model
            api_result = json.loads(response_content)
            # Update the payload with the model's response
            result_payload.update(api_result)
            return result_payload
        except Exception as e:
            logger.warning(f"API call failed on attempt {attempt + 1} for page {page_number}: {e}. Retrying in {2 ** attempt}s...")
            time.sleep(2 ** attempt)

    logger.error(f"API call failed after multiple retries for page {page_number}.")
    result_payload.update({
        "bucket": "api_error",
        "explanation": "API call failed after multiple retries."
    })
    return result_payload

def extract_and_structure_table(client: OpenAI, table_text: str) -> Dict[str, Any]:
    """
    Uses the AI to extract a title and structured data from table text.
    """
    sys_prompt = """
You are an expert data extraction assistant. Your task is to analyze the provided text, which contains one or more pages of a factor table from an insurance filing.

INSTRUCTIONS:
1.  **Identify the Table Title**: Find the most appropriate title for the table. This is often at the top of the first page. If no clear title exists, create a concise, descriptive one.
2.  **Recreate the Table**: Parse the text to reconstruct the table's data, including headers.
3.  **Handle Multi-Page Tables**: The text may come from multiple pages. You must intelligently stitch together table parts. The start of a new page might not have headers; in this case, you should continue the table from the previous page.
4.  **Return JSON**: Your output must be a single, valid JSON object with the following structure:
    {
      "table_title": "<The title you identified or created>",
      "table_data": [
        ["Header 1", "Header 2", "Header 3"],
        ["Row 1 Col 1", "Row 1 Col 2", "Row 1 Col 3"],
        ["Row 2 Col 1", "Row 2 Col 2", "Row 2 Col 3"]
      ]
    }
"""
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": table_text},
            ],
            model=MODEL,
            response_format={"type": "json_object"},
            temperature=0.0,
        )
        response_content = chat_completion.choices[0].message.content
        return json.loads(response_content)
    except Exception as e:
        logger.error(f"An error occurred during table extraction: {e}")
        return {
            "table_title": "Extraction Error",
            "table_data": [[f"An error occurred: {e}"]]
        }

def process_factor_tables(client: OpenAI, classified_df: pd.DataFrame, output_excel_path: str):
    """
    Finds, groups, and extracts factor tables into a multi-sheet Excel file.
    """
    logger.info("Starting Step 2: Processing 'factor_table' pages...")
    
    factor_table_pages = classified_df[classified_df['bucket'] == 'factor_table']
    if factor_table_pages.empty:
        logger.info("No pages were classified as 'factor_table'. Skipping table extraction.")
        return

    # Use ExcelWriter to save multiple sheets to one file
    with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
        # Group consecutive pages classified as factor tables
        page_indices = factor_table_pages.index
        if not page_indices.any():
            return
            
        current_group = [page_indices[0]]
        for i in range(1, len(page_indices)):
            # If the current page is not consecutive with the previous one, process the completed group
            if page_indices[i] != page_indices[i-1] + 1:
                process_table_group(client, classified_df, current_group, writer)
                current_group = []
            current_group.append(page_indices[i])
        
        # Process the last group
        if current_group:
            process_table_group(client, classified_df, current_group, writer)

    logger.info(f"Successfully saved extracted tables to '{output_excel_path}'")

def process_table_group(client: OpenAI, df: pd.DataFrame, group_indices: List[int], writer: pd.ExcelWriter):
    """
    Processes a single group of consecutive factor table pages.
    """
    page_numbers = df.loc[group_indices, 'page_number'].tolist()
    logger.info(f"Processing a potential table spanning pages: {page_numbers}")
    
    # Concatenate the text from all pages in the group
    combined_text = "\n--- NEW PAGE BREAK ---\n".join(df.loc[group_indices, 'text'])
    
    # Send the combined text to the AI for structuring
    structured_table = extract_and_structure_table(client, combined_text)
    
    table_title = structured_table.get("table_title", "Untitled Table")
    table_data = structured_table.get("table_data", [])
    
    if table_data:
        # Sanitize sheet title to be valid in Excel
        safe_sheet_name = "".join(c for c in table_title if c.isalnum() or c in (' ', '_')).rstrip()
        safe_sheet_name = safe_sheet_name[:31] # Max sheet name length is 31 chars
        
        # Convert the structured data to a DataFrame and save to a sheet
        table_df = pd.DataFrame(table_data[1:], columns=table_data[0])
        table_df.to_excel(writer, sheet_name=safe_sheet_name, index=False)
        logger.info(f"  - Saved table '{table_title}' to sheet '{safe_sheet_name}'")




In [13]:
# --- Main Script Execution ---
if __name__ == "__main__":
    # --- USER INPUT ---
    input_pdf_path = "./Inputs/PGR_Ohio_BNIC-134120828_trimmed.pdf"
    output_csv_path = "./Output/classified_pdf_text_trimmed_v3.csv"
    output_excel_path = "./Output/extracted_factor_tables.xlsx" # New output for tables

    # --- SCRIPT LOGIC ---
    # 1. Initialize OpenAI Client
    openai_client = get_openai_client(API_KEY_PATH)
    
    if openai_client:
        # 2. Extract text from PDF
        pdf_dataframe = extract_text_from_pdf(input_pdf_path)

        if not pdf_dataframe.empty:
            # 3. Classify each page
            logger.info("Starting Step 1: Page classification...")
            
            results = []
            total_pages = len(pdf_dataframe)
            for index, row in pdf_dataframe.iterrows():
                logger.info(f"Classifying page {row['page_number']}/{total_pages}...")
                result = classify_page_text(
                    client=openai_client, 
                    page_number=row['page_number'], 
                    page_text=row['text']
                )
                results.append(result)

            # 4. Create the final DataFrame from the list of result dictionaries
            final_df = pd.DataFrame(results)

            # 5. Save the classification results to CSV
            try:
                column_order = ['page_number', 'bucket', 'confidence', 'explanation', 'text']
                final_df_ordered = final_df.reindex(columns=column_order)
                
                final_df_ordered.to_csv(output_csv_path, index=False, encoding='utf-8')
                logger.info(f"\nSuccessfully saved classified text to '{output_csv_path}'")
                
                logger.info("\n--- Sample of Classification Data (text column omitted for brevity) ---")
                print(final_df_ordered.drop(columns=['text']).head().to_string())

            except Exception as e:
                logger.error(f"\nAn error occurred while saving the CSV file: {e}")
            
            # 6. Process the factor tables into an Excel file
            process_factor_tables(openai_client, final_df, output_excel_path)

2025-08-26 11:09:59,380 | INFO | Successfully loaded OpenAI API key.
2025-08-26 11:09:59,408 | INFO | Starting text extraction from 'PGR_Ohio_BNIC-134120828_trimmed.pdf'...
2025-08-26 11:09:59,421 | INFO | Found 72 pages in the document.
2025-08-26 11:10:00,163 | INFO |   - No text found on page 68.
2025-08-26 11:10:00,185 | INFO | Text extraction complete.
2025-08-26 11:10:00,185 | INFO | Starting Step 1: Page classification...
2025-08-26 11:10:00,186 | INFO | Classifying page 1/72...
2025-08-26 11:10:01,135 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-26 11:10:01,140 | INFO | Classifying page 2/72...
2025-08-26 11:10:01,806 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-26 11:10:01,818 | INFO | Classifying page 3/72...
2025-08-26 11:10:02,889 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-26 11:10:02,894 | INFO | Classifying page 4/72

   page_number             bucket  confidence                                                                         explanation
0            1  table of contents        0.95                      Lists sections, rules, forms, and attachments in the document.
1            2  intro information        0.90    Summary of filing requirements and administrative tracking information provided.
2            3  intro information        0.95  Contains filing overview, company details, and administrative information summary.
3            4  intro information        0.95             Contains company info, project details, and filing description summary.
4            5  intro information        0.90            Contains filing fees, company info, and administrative tracking details.


In [ ]:
-